In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import Utilities.myutil_regr as myutil

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [32]:
from pandas import Series

dataset = pandas.read_csv('dengue_labels_train.csv', usecols=[3], engine='python', skipfooter=3)

dataset1 = pandas.read_csv('dengue_features_train.csv')
dataset2 = pandas.read_csv('dengue_labels_train.csv')

iq, sj = myutil.split_dataset_by_city(dataset2)
iq = iq.drop(columns=['city', 'year', 'weekofyear'])
sj = sj.drop(columns=['city', 'year', 'weekofyear'])

dfall_iq, dfall_sj = myutil.split_dataset_by_city(dataset1)

dfall_iq = dfall_iq.drop(columns=['city', 'year', 'weekofyear', 'week_start_date'])
dfall_sj = dfall_sj.drop(columns=['city', 'year', 'weekofyear', 'week_start_date'])


dfall_iq['total_cases'] = Series(iq['total_cases'], index=dfall_iq.index)
dfall_sj['total_cases'] = Series(sj['total_cases'], index=dfall_iq.index)
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
dfall_iq.fillna(0, inplace=True)

In [54]:
# convert series to supervised learning
from pandas import DataFrame, concat
from sklearn.preprocessing import LabelEncoder

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


values = dfall_iq.values
# integer encode direction
print(values[:, 20])
encoder = LabelEncoder()
values[:,20] = encoder.fit_transform(values[:,20])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
#print (reframed)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]], axis=1, inplace=True)
#reframed.drop([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], axis=1, inplace=True)
print(type(reframed))


[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.
   0.   1.   1.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.
   0.   0.   0.   0.   1.   1.   0.   0.   1.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   1.   1.   1.   2.   4.   1.   4.  11.  16.  23.  12.  14.
  18.   8.   7.  10.   7.  10.   5.  11.   8.  18.  13.   9.  22.  10.
   5.  13.   2.  11.  11.   3.   7.   7.   4.   5.   6.   7.   7.   4.
   9.  17.   8.  22.  18.  21.  16.  31.  25.  28.  26.  18.  27.  11.
  38.  29.  21.  11.  10.   5.   6.   2.   1.   2.   2.   3.   5.   1.
   4.   2.   4.   0.   0.   0.   0.   1.   1.   1.   1.   1.   2.   3.
   4.   6.   2.   2.   5.   1.   1.   0.   0.   0.   0.   2.   0.   3.
   0.   0.   0.   2.   2.   3.   3.   3.   1.   2.   3.   6.   5.   1.
   4.   5.   8.   5.   2.   3.   3.   1.   6.   4.   1.   2.   3.   1.
   8. 

In [55]:
values = reframed.values
train = values
print (train.shape)

train_X, train_y = train[:, :-1], train[:, -1]

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [5]:
look_back = 1

trainX, trainY = create_dataset(dataset, look_back)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))

In [7]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=0)